In [1]:
import pandas as pd
in_file = "./soccer_2006.csv"
u18_df = pd.read_csv(in_file,encoding="UTF-8")

In [2]:
#列の順番を整理する time,playerの列を削除
u18_df=u18_df.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','url','results_away','results_home','goal_away','goal_home']]
u18_df.head(12)
# u18_df.shape

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home
0,NaN,高円宮杯第18回全日本ユース（U-15）サッカー選手権大会,2006,12,27,準決勝,SSSジュニアユース,FC東京U-15むさし,http://www.jfa.or.jp/archive/domestic/category...,3,0,"34分 梅内 和磨,58分 重松 健太郎,74分 年森 勝哉",NaN
1,NaN,高円宮杯第18回全日本ユース（U-15）サッカー選手権大会,2006,12,27,準決勝,FC東京U-15深川,ガンバ大阪ジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,4,1,"11分 望月 聖矢,44分 宇佐美 貴史,55分 原口 拓人,79分 高木 優太,","57分 中岡 哲志,"
2,NaN,高円宮杯第18回全日本ユース（U-15）サッカー選手権大会,2006,12,29,決勝,ガンバ大阪ジュニアユース,FC東京U-15むさし,http://www.jfa.or.jp/archive/domestic/category...,1,2,66分 年森 勝哉,"53分 大森 晃太郎,91分 宇佐美 貴史"
3,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home
4,57,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,25,準々決勝,横浜 Ｆ・マリノス ジュニアユース追浜,FC東京U-15深川,http://www.jfa.or.jp/archive/domestic/category...,3,1,"42分 阿部 巧,46分 笠原 健太,63分 山崎 直之",47分 小野 裕二
5,49,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,23,ラウンド16,FC前橋ジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,4,3,"76分 丸山 透 ,79分 岡 直樹 ,79分 中島 明人 ,99分 小野 悠斗","2分 小林 雄剛 ,6分 糸井 康裕 ,47分 高橋 大樹"
6,58,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,25,準々決勝,ガンバ大阪 ジュニアユース,清水エスパルス ジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,4,"58分 深澤 諄也,71分 青木 達也","11分 宇佐美 貴史,15分 宇佐美 貴史,42分 原口 拓人,72分 明智 裕典"
7,33,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,17,1次ラウンド第3戦,日章学園中学校,FC前橋ジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,0,"9分 高橋 大樹,65分 高橋 大樹",NaN
8,50,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,23,ラウンド16,FC東京U-15深川,柏レイソルU-15,http://www.jfa.or.jp/archive/domestic/category...,1,2,39分 橋本 拓門,"23分 高橋 寛太 ,31分 藪下 阿由武"
9,59,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,25,準々決勝,サンフレッチェ広島F.C ジュニアユース,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,1,1,NaN,79分 茶島 雄介


In [3]:
# スコアがNone(=未実施)の試合の行を削除
u18_df=u18_df[u18_df['results_away'] != "None"]
u18_df.shape

(64, 13)

In [4]:
# 年が変わるときの行(カラム名が入っている)を削除(一応...)
u18_df=u18_df[u18_df['results_away'] != "results_away"]
u18_df.shape

(63, 13)

In [5]:
# 抽出(任意)
# u18_df=u18_df[u18_df['year'] == int("2013")]
u18_df = u18_df[u18_df["id"].isnull() != True ]

# u18_df

In [6]:
# # yearに"2002"が入ってしまっているものの対処(未実装)
# u18_df=u18_df[u18_df['year'].str.match('.*,2002')!= "True"]

# u18_df.shape

In [7]:
#idの混同を避けるため、idとyearを合わせる
u18_df["merg_id"] = u18_df["year"].astype(str)+u18_df["id"].astype(str)
u18_df["merg_id"] = u18_df["merg_id"].astype(int)
u18_df.dtypes

id              object
leagu_name      object
year            object
month           object
day             object
round           object
team_home       object
team_away       object
url             object
results_away    object
results_home    object
goal_away       object
goal_home       object
merg_id          int32
dtype: object

In [8]:
# ここからaway

In [9]:
#awayチームの得点者とその試合のidのみを抜き出す
df_spr_away = pd.DataFrame(u18_df['goal_away'].str.split(',', expand=True))
df_spr_away["merg_id"] =u18_df["merg_id"]
df_spr_away.head()

,0,1,2,3,4,5,6,merg_id
4,42分 阿部 巧,46分 笠原 健太,63分 山崎 直之,None,None,None,None,200657
5,76分 丸山 透,79分 岡 直樹,79分 中島 明人,99分 小野 悠斗,None,None,None,200649
6,58分 深澤 諄也,71分 青木 達也,None,None,None,None,None,200658
7,9分 高橋 大樹,65分 高橋 大樹,None,None,None,None,None,200633
8,39分 橋本 拓門,None,None,None,None,None,None,200650


In [10]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_away0 = pd.DataFrame({
    "goal":df_spr_away[0],
    "merg_id":df_spr_away["merg_id"]
})
df_spr_away0=df_spr_away0[df_spr_away0['goal'] != "None"] 
df_spr_away0.dropna(inplace=True) #はもともと"None"なので、使えない
df_spr_away0["count_away"]=int(1)
df_spr_away0.head(3)

,goal,merg_id,count_away
4,42分 阿部 巧,200657,1
5,76分 丸山 透,200649,1
6,58分 深澤 諄也,200658,1


In [11]:
#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_away.columns)) -1) ):
    df_spr_away_n = pd.DataFrame({
    "goal":df_spr_away[repeat],
    "merg_id":df_spr_away["merg_id"],
    })
    df_spr_away_n.dropna(inplace=True)
    df_spr_away_n["count_away"]=int(1+repeat)
    df_spr_away0 = pd.concat([df_spr_away0,df_spr_away_n],sort=True,copy=False,ignore_index=True)

df_spr_away0

,count_away,goal,merg_id
0,1,42分 阿部 巧,200657
1,1,76分 丸山 透,200649
2,1,58分 深澤 諄也,200658
3,1,9分 高橋 大樹,200633
4,1,39分 橋本 拓門,200650
5,1,5分 半沢 大典,200641
6,1,20分 山﨑 翼,200634
7,1,49分 岡部 亮誠,200651
8,1,29分 鈴木 恵士,200625
9,1,15分 宮吉 拓実,200642


In [12]:
# 空白やNoneを消す
df_spr_away0 = df_spr_away0[df_spr_away0["goal"].isnull() != True ]
df_spr_away0 = df_spr_away0[df_spr_away0['goal'].str.contains("分") != False]

df_spr_away0.shape

(96, 3)

In [13]:
#選手名と時間を分ける
df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_away0.rename(columns={0: 'time', 1: 'player'}, inplace=True)

# additional timeで別カラム
try:
    df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["time"].str.split("+",expand=True)],axis=1).drop("time", axis=1).fillna(-1)
    df_spr_away0.rename(columns={0: 'n_time', 1: 'a_time'}, inplace=True)
    df_spr_away0["n_time"] =df_spr_away0["n_time"].astype(int)
    df_spr_away0["a_time"] =df_spr_away0["a_time"].astype(int)
except:
    df_spr_away0["n_time"] =df_spr_away0["n_time"].astype(int)
    df_spr_away0["a_time"] =int("-1")
df_spr_away0.head()

,count_away,merg_id,player,n_time,a_time
0,1,200657,阿部 巧,42,-1
1,1,200649,丸山 透,76,-1
2,1,200658,深澤 諄也,58,-1
3,1,200633,高橋 大樹,9,-1
4,1,200650,橋本 拓門,39,-1


In [14]:
# 名前に「分」が入ってる人
def inc_hunn(df):
    try:
        df.loc[ (df[3] != -1 ), "player"] = df["player"].astype(str) + str("分") + df[2].astype(str)
        del df[3]
    except:
        try:
            df.loc[ (df[2] != -1 ), "player"] = df["player"].astype(str) + str("分") + df[2].astype(str)
            del df[2]
        except:
            pass
inc_hunn(df_spr_away0)

In [15]:
#ここからhome

In [35]:
#homeチームの得点者とその試合のidのみを抜き出す
df_spr_home = pd.DataFrame(u18_df['goal_home'].str.split(',', expand=True))
df_spr_home["merg_id"] =u18_df["merg_id"]

In [36]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_home0 = pd.DataFrame({
    "goal":df_spr_home[0],
    "merg_id":df_spr_home["merg_id"]
})
df_spr_home0=df_spr_home0[df_spr_home0['goal'] != "None"] # df_spr_home0.dropna(inplace=True) はもともと"None"なので、使えない
df_spr_home0["count_home"]=int(1)

#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_home.columns)) -1) ):
    df_spr_home_n = pd.DataFrame({
    "goal":df_spr_home[repeat],
    "merg_id":df_spr_home["merg_id"],
    })
    df_spr_home_n.dropna(inplace=True)
    df_spr_home_n["count_home"]=int(1+repeat)
    df_spr_home0 = pd.concat([df_spr_home0,df_spr_home_n],sort=True,copy=False,ignore_index=True)

df_spr_home0

,count_home,goal,merg_id
0,1,47分 小野 裕二,200657
1,1,2分 小林 雄剛,200649
2,1,11分 宇佐美 貴史,200658
3,1,NaN,200633
4,1,23分 高橋 寛太,200650
5,1,79分 茶島 雄介,200659
6,1,20分 伊勢 隆司,200617
7,1,41分 藤浪 智大,200641
8,1,NaN,200634
9,1,16分 大森 晃太郎,200651


In [37]:
# 空白やNoneを消す
df_spr_home0 = df_spr_home0[df_spr_home0["goal"].isnull() != True ]
df_spr_home0 = df_spr_home0[df_spr_home0['goal'].str.contains("分") != False]

In [38]:
#選手名と時間を分ける
df_spr_home0 = pd.concat([df_spr_home0,df_spr_home0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_home0.rename(columns={0: 'time', 1: 'player'}, inplace=True)

try:
    df_spr_home0 = pd.concat([df_spr_home0,df_spr_home0["time"].str.split("+",expand=True)],axis=1).drop("time", axis=1).fillna(-1)
    df_spr_home0.rename(columns={0: 'n_time', 1: 'a_time'}, inplace=True)
    df_spr_home0["n_time"] =df_spr_home0["n_time"].astype(int)
    df_spr_home0["a_time"] =df_spr_home0["a_time"].astype(int)
except:
    df_spr_home0["n_time"] =df_spr_home0["n_time"].astype(int)
    df_spr_home0["a_time"] =int("-1")

df_spr_home0

,count_home,merg_id,player,n_time,a_time
0,1,200657,小野 裕二,47,-1
1,1,200649,小林 雄剛,2,-1
2,1,200658,宇佐美 貴史,11,-1
4,1,200650,高橋 寛太,23,-1
5,1,200659,茶島 雄介,79,-1
6,1,200617,伊勢 隆司,20,-1
7,1,200641,藤浪 智大,41,-1
9,1,200651,大森 晃太郎,16,-1
10,1,200660,山口 潤,30,-1
12,1,200642,小野 悠斗,66,-1


In [39]:
# 名前に「分」が入ってる人(いない場合はエラー、無視して次に進む)
inc_hunn(df_spr_home0)

In [40]:
#ここからまとめて

In [46]:
#homeとawayを合体 #並び変える（ソート）
df_merge = pd.concat([df_spr_away0,df_spr_home0],sort=True,copy=False,ignore_index=True)
#試合情報を結合
df_merged = pd.merge(df_merge,u18_df, on="merg_id").sort_values(["year","id","n_time","a_time"],ascending=[False,True,True,True]).reset_index(drop=True)

df_merged["differ"] =df_merged["merg_id"].diff().fillna(1)
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,NaN,1.0,200601,37,飯山 成周,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",1.0
1,-1,NaN,2.0,200601,39,清水 敬裕,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
2,-1,NaN,3.0,200601,44,伊勢 隆司,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
3,-1,1.0,NaN,200601,62,根岸 泰浩,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
4,-1,1.0,NaN,200602,30,小野 裕二,02,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,愛媛FCジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,1,0,30分小野 裕二,NaN,1.0
5,-1,NaN,1.0,200603,5,河本 力,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",1.0
6,-1,NaN,2.0,200603,7,篠崎 翔太,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",0.0
7,-1,NaN,3.0,200603,77,菅原 拓哉,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",0.0
8,-1,NaN,1.0,200604,57,時任 郁弥,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,NaN,"57分時任 郁弥,67分福留 真太郎,72分時任 郁弥,79分金丸 誠豪",1.0
9,-1,NaN,2.0,200604,67,福留 真太郎,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,NaN,"57分時任 郁弥,67分福留 真太郎,72分時任 郁弥,79分金丸 誠豪",0.0


In [47]:
#idが変わる試合 かつ counthomeがnullのもの(=相手が先制しているので、counthomeは0になるべき)
df_merged.loc[(df_merged["differ"] != 0.0) &  (df_merged["count_home"].isnull() ), "count_home"] = 0
#awayも同様
df_merged.loc[(df_merged["differ"] != 0.0) &  (df_merged["count_away"].isnull() ), "count_away"] = 0

df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,0.0,1.0,200601,37,飯山 成周,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",1.0
1,-1,NaN,2.0,200601,39,清水 敬裕,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
2,-1,NaN,3.0,200601,44,伊勢 隆司,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
3,-1,1.0,NaN,200601,62,根岸 泰浩,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
4,-1,1.0,0.0,200602,30,小野 裕二,02,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,愛媛FCジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,1,0,30分小野 裕二,NaN,1.0
5,-1,0.0,1.0,200603,5,河本 力,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",1.0
6,-1,NaN,2.0,200603,7,篠崎 翔太,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",0.0
7,-1,NaN,3.0,200603,77,菅原 拓哉,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",0.0
8,-1,0.0,1.0,200604,57,時任 郁弥,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,NaN,"57分時任 郁弥,67分福留 真太郎,72分時任 郁弥,79分金丸 誠豪",1.0
9,-1,NaN,2.0,200604,67,福留 真太郎,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,NaN,"57分時任 郁弥,67分福留 真太郎,72分時任 郁弥,79分金丸 誠豪",0.0


In [48]:
#残りのNaNは、相手ゴール分なので、一つ上の行のデータで埋める
df_merged["count_away"]=df_merged["count_away"].fillna(method='ffill')
df_merged["count_home"]=df_merged["count_home"].fillna(method='ffill')
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,0.0,1.0,200601,37,飯山 成周,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",1.0
1,-1,0.0,2.0,200601,39,清水 敬裕,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
2,-1,0.0,3.0,200601,44,伊勢 隆司,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
3,-1,1.0,3.0,200601,62,根岸 泰浩,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,62分根岸 泰浩,"37分飯山 成周,39分清水 敬裕,44分伊勢 隆司",0.0
4,-1,1.0,0.0,200602,30,小野 裕二,02,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,愛媛FCジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,1,0,30分小野 裕二,NaN,1.0
5,-1,0.0,1.0,200603,5,河本 力,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",1.0
6,-1,0.0,2.0,200603,7,篠崎 翔太,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",0.0
7,-1,0.0,3.0,200603,77,菅原 拓哉,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,NaN,"5分河本 力,7分篠崎 翔太,77分菅原 拓哉",0.0
8,-1,0.0,1.0,200604,57,時任 郁弥,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,NaN,"57分時任 郁弥,67分福留 真太郎,72分時任 郁弥,79分金丸 誠豪",1.0
9,-1,0.0,2.0,200604,67,福留 真太郎,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,NaN,"57分時任 郁弥,67分福留 真太郎,72分時任 郁弥,79分金丸 誠豪",0.0


In [49]:
df_merged["goal_away"]=df_merged["count_away"].astype(int)
df_merged["goal_home"]=df_merged["count_home"].astype(int)

df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,0.0,1.0,200601,37,飯山 成周,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,1,1.0
1,-1,0.0,2.0,200601,39,清水 敬裕,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,2,0.0
2,-1,0.0,3.0,200601,44,伊勢 隆司,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,3,0.0
3,-1,1.0,3.0,200601,62,根岸 泰浩,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,1,3,0.0
4,-1,1.0,0.0,200602,30,小野 裕二,02,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,愛媛FCジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,1,0,1,0,1.0
5,-1,0.0,1.0,200603,5,河本 力,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,1,1.0
6,-1,0.0,2.0,200603,7,篠崎 翔太,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,2,0.0
7,-1,0.0,3.0,200603,77,菅原 拓哉,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,3,0.0
8,-1,0.0,1.0,200604,57,時任 郁弥,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,0,1,1.0
9,-1,0.0,2.0,200604,67,福留 真太郎,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,0,2,0.0


In [50]:
#additional timeをプラス表記に
df_merged["time"] = df_merged["n_time"]
df_merged.loc[ (df_merged["a_time"] !=-1 ), "time"] = df_merged["n_time"].astype(str) + str("+") + df_merged["a_time"].astype(str)

#列の整理
df_merged=df_merged.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','url','results_away','results_home','goal_away','goal_home','time','player']]

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,time,player
0,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,1,37,飯山 成周
1,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,2,39,清水 敬裕
2,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,3,44,伊勢 隆司
3,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,1,3,62,根岸 泰浩
4,02,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,愛媛FCジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,1,0,1,0,30,小野 裕二
5,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,1,5,河本 力
6,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,2,7,篠崎 翔太
7,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,3,77,菅原 拓哉
8,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,0,1,57,時任 郁弥
9,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,0,2,67,福留 真太郎


In [51]:
df_merged.head(20)

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,time,player
0,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,1,37,飯山 成周
1,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,2,39,清水 敬裕
2,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,0,3,44,伊勢 隆司
3,01,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,日章学園中学校,帝京大学可児中学校,http://www.jfa.or.jp/archive/domestic/category...,1,3,1,3,62,根岸 泰浩
4,02,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,愛媛FCジュニアユース,横浜F･マリノスジュニアユース追浜,http://www.jfa.or.jp/archive/domestic/category...,1,0,1,0,30,小野 裕二
5,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,1,5,河本 力
6,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,2,7,篠崎 翔太
7,03,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,スポーツクラブ岐阜VAMOS,SSSジュニアユース,http://www.jfa.or.jp/archive/domestic/category...,2,1,0,3,77,菅原 拓哉
8,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,0,1,57,時任 郁弥
9,04,高円宮杯 第18回全日本ユース（U-15）サッカー選手権大会,2006,12,10,1次ラウンド第1戦,宮崎日本大学中学校,青森山田中学校,http://www.jfa.or.jp/archive/domestic/category...,0,4,0,2,67,福留 真太郎


In [27]:
# 出力CSVの名前
from pathlib import Path
# newname = Path(in_file).stem
newname = "u12_2013"
print (newname)

u12_2013


In [28]:
# タイムスタンプ
import os
from datetime import datetime
t = os.path.getmtime(in_file)
# エポック秒をdatetimeに変換
dt = datetime.fromtimestamp(t)
datadate = dt.strftime('%Y%m%d%H%M')
print (datadate)

201911040034


In [29]:
# 出力
df_merged.to_csv("./adj_" + newname + "_" + datadate + ".csv", 
          index=False   # 行インデックスを削除
         )

In [4]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'dataformat.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'dataformat.ipynb'], returncode=0)